HyperTS内置的模型包含一些超参数，通常它们被固定或者通过搜索者从搜索空间中优化。在一些情况下，我们也需要一些专家经验来调控某些参数，例如epochs, batch_size, learning_rate等来定向获得更好的性能或者提升训练速度。

为了解决这个问题，我们可以通过将这些参数传入到``run()``函数中，从而实现非侵入的调控参数。

**使用示例:**

#### 1. 准备数据

In [1]:
from hyperts.datasets import load_network_traffic
from sklearn.model_selection import train_test_split

In [2]:
df = load_network_traffic(univariate=True)
train_data, test_data = train_test_split(df, test_size=168, shuffle=False)

#### 2. 创建实验并运行它

In [3]:
from hyperts import make_experiment

In [4]:
experiment = make_experiment(train_data=train_data.copy(),
                             task='forecast',
                             mode='dl',
                             timestamp='TimeStamp',
                             covariates=['HourSin', 'WeekCos', 'CBWD'],
                             forecast_train_data_periods=24*12,
                             max_trials=1)

In [5]:
model = experiment.run(epochs=100,
                       final_train_epochs=200,
                       batch_size=32, 
                       learning_rate=0.01,
                       dl_forecast_window=48)

In [6]:
model.get_pipeline_params()

<bound method Pipeline.get_params of Pipeline(steps=[('data_preprocessing',
                 TSFDataPreprocessStep(covariate_cleaner=CovariateTransformer(covariables=['HourSin',
                                                                                           'WeekCos',
                                                                                           'CBWD'],
                                                                              data_cleaner_args={'correct_object_dtype': False,
                                                                                                 'int_convert_to': 'str'}),
                                       covariate_cleaner__covariables=['HourSin',
                                                                       'WeekCos',
                                                                       'CBWD'],
                                       covariate_cleaner__data_cleaner_args={'correct_object_dtype': False,
                

#### 3. 推理与评估

In [7]:
X_test, y_test = model.split_X_y(test_data.copy())
forecast = model.predict(X_test)
results = model.evaluate(y_true=y_test, y_pred=forecast)
results

,Metirc,Score
0,mae,80572.3467
1,mse,14294828393.1990
2,rmse,119560.9819
3,mape,1.0549
4,smape,0.5751
